# 自定义指令微调数据集

## 数据准备
OpenAI SFT数据格式

In [ ]:
[{
    "messages": [
        {"role": "system", "content": "xxx."},
        {"role": "user", "content": "xxx."},
        {"role": "assistant", "content": "xxx."}
    ]
},
    {
    "messages": [
        {"role": "system", "content": "xxx."},
        {"role": "user", "content": "xxx."},
        {"role": "assistant", "content": "xxx.", "loss": False},
        {"role": "user", "content": "xxx."},
        {"role": "assistant", "content": "xxx.", "loss": True}
    ]
}]

loss字段，用于控制某轮assistant的输出不计算loss

system 和 user 的 loss 默认为 False

assistant 的 loss 默认为 True

## 训练

### 导出config
xtuner copy-cfg internlm2_chat_7b_qlora_custom_sft_e1 .

### 修改config

```
- data_files = ['/path/to/json/file.json']
+ data_files = ['/path/to/custom_sft1.json', '/path/to/custom_sft2.json', ...]
```

或者使用某个目录下所有的 json 文件作为训练数据集

In [ ]:
#######################################################################
#                          PART 1  Settings                           #
#######################################################################
# Data
- data_files = ['/path/to/json/file.json']
+ data_dir = '/dir/to/custom_sft'

#######################################################################
#                      PART 3  Dataset & Dataloader                   #
#######################################################################
train_dataset = dict(
-   dataset=dict(type=load_dataset, path='json', data_files=data_files),
+   dataset=dict(type=load_dataset, path='json', data_dir=data_dir),
    ...)

lora训练
```
#######################################################################
#                      PART 2  Model & Tokenizer                      #
#######################################################################
model = dict(
    type=SupervisedFinetune,
    use_varlen_attn=use_varlen_attn,
    llm=dict(
        type=AutoModelForCausalLM.from_pretrained,
        pretrained_model_name_or_path=pretrained_model_name_or_path,
        trust_remote_code=True,
        torch_dtype=torch.float16,
-       quantization_config=dict(
-           type=BitsAndBytesConfig,
-           load_in_4bit=True,
-           load_in_8bit=False,
-           llm_int8_threshold=6.0,
-           llm_int8_has_fp16_weight=False,
-           bnb_4bit_compute_dtype=torch.float16,
-           bnb_4bit_use_double_quant=True,
-           bnb_4bit_quant_type='nf4')
    ),
    lora=dict(
        type=LoraConfig,
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias='none',
        task_type='CAUSAL_LM'))
```

全量参数训练
```
#######################################################################
#                      PART 2  Model & Tokenizer                      #
#######################################################################
model = dict(
    type=SupervisedFinetune,
    use_varlen_attn=use_varlen_attn,
    llm=dict(
        type=AutoModelForCausalLM.from_pretrained,
        pretrained_model_name_or_path=pretrained_model_name_or_path,
        trust_remote_code=True,
        torch_dtype=torch.float16,
-       quantization_config=dict(
-           type=BitsAndBytesConfig,
-           load_in_4bit=True,
-           load_in_8bit=False,
-           llm_int8_threshold=6.0,
-           llm_int8_has_fp16_weight=False,
-           bnb_4bit_compute_dtype=torch.float16,
-           bnb_4bit_use_double_quant=True,
-           bnb_4bit_quant_type='nf4')
    ),
-   lora=dict(
-       type=LoraConfig,
-       r=64,
-       lora_alpha=16,
-       lora_dropout=0.1,
-       bias='none',
-       task_type='CAUSAL_LM')
)
```

## 训练

In [ ]:
NPROC_PER_NODE=8 xtuner train internlm2_chat_7b_qlora_custom_sft_e1_copy.py --deepspeed deepspeed_zero1

# 训练日志及 checkpoint 将默认保存在 ./work_dirs/，可以通过命令 xtuner train --work-dir ${SAVE_PATH} 指定保存路径。

## 模型转换
```
xtuner convert pth_to_hf ${FINETUNE_CFG} ${PTH_PATH} ${SAVE_PATH}
# 例如：xtuner convert pth_to_hf internlm2_chat_7b_qlora_custom_sft_e1_copy.py ./iter_2000.pth ./iter_2000_hf
```

## 对话
```
xtuner chat ${NAME_OR_PATH_TO_LLM} --adapter {NAME_OR_PATH_TO_ADAPTER} --prompt-template ${PROMPT_TEMPLATE} [optional arguments]
# 例如：xtuner chat internlm/internlm2-7b --adapter ./iter_2000_hf --prompt-template internlm2_chat
```
全量参数训练
```
xtuner chat ${PATH_TO_LLM} --prompt-template ${PROMPT_TEMPLATE} [optional arguments]
# 例如：xtuner chat ./iter_2000_hf --prompt-template internlm2_chat
```
其中 ${PROMPT_TEMPLATE} 表示模型的对话模板，需要与训练用的 config 中的 prompt_template 字段保持一致

## 模型合并
```
xtuner convert merge ${LLM} ${LLM_ADAPTER} ${SAVE_PATH}
```

## 评测
OpenCompass  
https://github.com/InternLM/opencompass